Experimentos y ejecuciones en modelos de aprendizaje automático
===============================================================

## Introducción

El seguimiento (o tracking) de experimentos de aprendizaje automático es el proceso de organizar, registrar y analizar los resultados de los procesos de entrenamiento y evaluación de modelos. El objetivo es obtener información que pueda usarse para mejorar modelos futuros y tomar mejores decisiones sobre qué algoritmos y parámetros a usar.

### Beneficios

Hay muchos beneficios al usar herramientas de tracking de experimentos de aprendizaje automático, particularmente:

- Ayuda a comprender el rendimiento de los modelos.
- Ayuda a comparar differentes modelos y encontrar los hiperparámetros correctos.
- Posibilita la automatización de experimentos.
- Posibilita la reproducibilidad de los resultados.

### Conceptos

La mayoria de las herramientas de seguimiento utilizan 6 conceptos fundamentales:

- **Experimentos:** Representan el problema que queremos resolver. Los experimentos en general se identifican con un nombre y sirven de contenedor de todas las ejecuciones de nuestras iteraciones de aprendizaje automático.
- **Ejecuciones (runs o trials):** Es cada de la iteraciones dentro del proceso de desarrollo, incluyendo el proceso de preprocesamiento de datos, entrenamiento de modelos, y evaluación. En simples terminos, cualquier operación que es necesaria para generar y/o evaluar nuestro modelo de aprendizaje automático se sucede dentro de una ejecución.
- **Parametros:** Representan cada uno de los valores que nuestro proceso de aprendizaje automático utiliza cómo configuración para el proceso de aprendizaje. Los hiperparámetros son casos típicos.
- **Métricas:** Representan cada una de las métricas que podemos utilizar para comparar o evaluar los resultados de la ejecución.
- **Modelos:** Representan cada uno de los modelos de aprendizaje automático que fueron generados durante la ejecución.
- **Artefactos:** Cualquier otro archivo que es generado como parte del proceso y que nos interesa mantener bajo control (source control) a los efectos de auditoria y reproducibilidad.


### Herramientas

Existen multiples herramientas en el mercado para el siguimiento de experimentos. Algunas de las más populares incluye:

- Herramientas incorporadas en nubes públicas:

  - Azure Machine Learning
  - AWS SageMaker
  - GCP Vertex AI

- Herramientas especificas:

  - MLflow
  - Weights & Biases
  - Comet
  - ClearML

En este ejemplo, utilizaremos la herramienta Comet por el simple hecho de que es gratuita y tiene una curva de aprendizaje muy baja.

## Seguimiento del proceso de entrenando para el problema censo de la UCI

### Instalación

Instalemos las librerias que necesitamos para resolver nuestro problema:

In [1]:
!wget https://raw.githubusercontent.com/santiagxf/E72102/master/docs/develop/modeling/experimentation/code/tuning_bayesian.txt \
    --quiet --no-clobber
!pip install -r tuning_bayesian.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 3.3 MB/s eta 0:00:00


Adicionalmente, instalaremos la libreria de Comet:

In [2]:
%pip install comet_ml --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 534.7/534.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.1/510.1 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.2 MB/s eta 

### Sobre el conjunto de datos del censo UCI

El conjunto de datos del censo de la UCI es un conjunto de datos en el que cada registro representa a una persona. Cada registro contiene 14 columnas que describen a una una sola persona, de la base de datos del censo de Estados Unidos de 1994. Esto incluye información como la edad, el estado civil y el nivel educativo. La tarea es determinar si una persona tiene un ingreso alto (definido como ganar más de $50 mil al año). Esta tarea, dado el tipo de datos que utiliza, se usa a menudo en el estudio de equidad, en parte debido a los atributos comprensibles del conjunto de datos, incluidos algunos que contienen tipos sensibles como la edad y el género, y en parte también porque comprende una tarea claramente del mundo real.

Descargamos el conjunto de datos

In [3]:
!wget https://santiagxf.blob.core.windows.net/public/datasets/uci_census.zip \
    --quiet --no-clobber
!mkdir -p datasets/uci_census
!unzip -qq uci_census.zip -d datasets/uci_census

Lo importamos

In [16]:
import pandas as pd
import numpy as np

train = pd.read_csv('datasets/uci_census/data/adult-train.csv')
test = pd.read_csv('datasets/uci_census/data/adult-test.csv')

### Configurando Comet en nuestro notebook

In [17]:
import comet_ml

experiment = comet_ml.Experiment(
    api_key="XXXXXXXXXXX",
    project_name = "uci-adults-income",
    workspace="santiagxf"
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/santiagxf/uci-adults-income/b3d5899ad13b4fefa6da6498072a5ae1



### Creando el modelo

En este ejemplo, crearemos un modelo muy sencillo con el cual trabajar.

Preparando nuestros conjuntos de datos

In [18]:
X_train = train.drop(['income'], axis=1)
y_train = train['income'].to_numpy()
X_test = test.drop(['income'], axis=1)
y_test = test['income'].to_numpy()

In [19]:
classes = train['income'].unique().tolist()
features = X_train.columns.values.tolist()
categorical_features = X_train.dtypes[X_train.dtypes == 'object'].index.tolist()

Realizaremos un pequeño preprocesamiento antes de entrenar el modelo:

- Imputaremos los valores faltantes de las caracteristicas numéricas con la media
- Imputaremos los valores faltantes de las caracteristicas categóricas con el valor `?`
- Escalaremos los valores numericos utilizando un `StandardScaler`
- Codificaremos las variables categóricas utilizando `OneHotEncoder`

In [20]:
from typing import Tuple, List

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


def prepare(X: pd.DataFrame) -> Tuple[np.ndarray, sklearn.compose.ColumnTransformer]:
    pipe_cfg = {
        'num_cols': X.dtypes[X.dtypes == 'int64'].index.values.tolist(),
        'cat_cols': X.dtypes[X.dtypes == 'object'].index.values.tolist(),
    }

    num_pipe = Pipeline([
        ('num_imputer', SimpleImputer(strategy='median')),
        ('num_scaler', StandardScaler())
    ])

    cat_pipe = Pipeline([
        ('cat_imputer', SimpleImputer(strategy='constant', fill_value='?')),
        ('cat_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    transformations = ColumnTransformer([
        ('num_pipe', num_pipe, pipe_cfg['num_cols']),
        ('cat_pipe', cat_pipe, pipe_cfg['cat_cols'])
    ])
    X = transformations.fit_transform(X)

    return X, transformations


X_train_transformed, transformations = prepare(X_train)
X_test_transformed = transformations.transform(X_test)

Entrenamos un modelo basado en `lightgbm`

In [21]:
from lightgbm import LGBMClassifier

with experiment.train():
    clf = LGBMClassifier(n_estimators=5)
    model = clf.fit(X_train_transformed, y_train)

Podemos revisar la performance del modelo:

In [23]:
from sklearn.metrics import classification_report

with experiment.test():
    predictions = model.predict(X_test_transformed)
    print(classification_report(y_test, predictions))
    for key, value in classification_report(y_test, predictions, output_dict=True).items():
        if key == "accuracy":
            experiment.log_metric(key, value)
        else:
            experiment.log_metrics(value, prefix=f"{key}")

              precision    recall  f1-score   support

       <=50K       0.81      1.00      0.90     12435
        >50K       0.98      0.25      0.40      3846

    accuracy                           0.82     16281
   macro avg       0.90      0.63      0.65     16281
weighted avg       0.85      0.82      0.78     16281



### Finalizando la ejecución del experimento

In [24]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/santiagxf/uci-adults-income/b3d5899ad13b4fefa6da6498072a5ae1
COMET INFO:   Metrics:
COMET INFO:     test_<=50K_f1-score         : 0.8955966677485665
COMET INFO:     test_<=50K_precision        : 0.8118870145154963
COMET INFO:     test_<=50K_recall           : 0.9985524728588661
COMET INFO:     test_<=50K_support          : 12435
COMET INFO:     test_>50K_f1-score          : 0.4009931719428926
COMET INFO:     test_>50K_precision         : 0.9817629179331308
COMET INFO:     test_>50K_recall            : 0.25195007800312014
COMET INFO:     test_>50K_support           : 3846
COMET INFO:     test_accuracy               : 0.8221853694